# Recunoașterea Entităților Denumite (NER)

Acest notebook face parte din [Curriculum AI pentru Începători](http://aka.ms/ai-beginners).

În acest exemplu, vom învăța cum să antrenăm un model NER pe [Corpus Anotat pentru Recunoașterea Entităților Denumite](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) din Kaggle. Înainte de a continua, vă rugăm să descărcați fișierul [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) în directorul curent.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Pregătirea setului de date

Vom începe prin a citi setul de date într-un dataframe. Dacă vrei să afli mai multe despre utilizarea Pandas, vizitează o [lecție despre procesarea datelor](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) din [Data Science pentru Începători](http://aka.ms/datascience-beginners).


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Să obținem etichete unice și să creăm dicționare de căutare pe care le putem folosi pentru a converti etichetele în numere de clasă:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Acum trebuie să facem același lucru cu vocabularul. Pentru simplitate, vom crea vocabularul fără a ține cont de frecvența cuvintelor; în viața reală, s-ar putea să doriți să utilizați vectorizatorul Keras și să limitați numărul de cuvinte.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Trebuie să creăm un set de date de propoziții pentru antrenament. Să parcurgem setul de date original și să separăm toate propozițiile individuale în `X` (liste de cuvinte) și `Y` (liste de tokeni):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Pentru simplitate, vom completa toate propozițiile cu 0 tokeni până la lungimea maximă. În viața reală, s-ar putea să dorim să folosim o strategie mai inteligentă și să completăm secvențele doar în cadrul unui singur minibatch.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Definirea Rețelei de Clasificare a Token-urilor

Vom folosi o rețea bidirecțională LSTM cu două straturi pentru clasificarea token-urilor. Pentru a aplica un clasificator dens fiecărei ieșiri a ultimului strat LSTM, vom utiliza construcția `TimeDistributed`, care replică același strat dens pentru fiecare dintre ieșirile LSTM la fiecare pas:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Rețineți aici că specificăm explicit `maxlen` pentru setul nostru de date - în cazul în care dorim ca rețeaua să poată gestiona secvențe de lungimi variabile, trebuie să fim puțin mai ingenioși când definim rețeaua.

Să antrenăm acum modelul. Pentru viteză, vom antrena doar pentru o epocă, dar puteți încerca să antrenați pentru o perioadă mai lungă. De asemenea, poate doriți să separați o parte din setul de date ca set de date de antrenament, pentru a observa acuratețea validării.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Testarea Rezultatelor

Să vedem acum cum funcționează modelul nostru de recunoaștere a entităților pe o propoziție exemplu:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Concluzie

Chiar și un model LSTM simplu oferă rezultate rezonabile pentru NER. Totuși, pentru a obține rezultate mult mai bune, este recomandat să folosești modele mari de limbaj pre-antrenate, cum ar fi BERT. Antrenarea BERT pentru NER utilizând biblioteca Huggingface Transformers este descrisă [aici](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
